In [15]:


import pandas as pd
import urllib 
from bs4 import BeautifulSoup
import csv
import re
import datetime
import time
import random
from urllib.parse import urlparse

#value="1">Alumni</option>
#categorydict=value="2">Art</option>
#value="3">Career</option>
#value="4">Clinic</option>
#value="5">Film</option>
#value="15">International/Global</option>
#value="6">Meeting</option>
#value="7">Multicultural</option>
# value="16">Other</option>
# value="8">Performance</option>
#value="9">Religious</option>
#value="10">Special Event</option>
# value="11">Sporting Event</option>
#value="12">Talk, Lecture, Seminar</option>
#value="13">Testing</option>
#value="14">Workshop, Course</option>
categorydict = {
            2: 'Art',
            3: 'Career',
            4: 'Clinic',
            5: 'Film',
            6: 'Meeting',
            7: 'Multicultural',
            8: 'Performance',
            9: 'Religious',
            10: 'Special Event',
            11: 'Sporting Event',
            12: 'Talk, Lecture, Seminar',
            13: 'Testing',
            14: 'Workshop, Course',
            15: 'International/Global',
            16:'Other'
    
}
#https://ruevents.rutgers.edu/events/displayEventList.html?region=S&_targetAudienceList[11].selected=on&_targetAudienceList[4].selected=on&_targetAudienceList[9].selected=on&targetAudienceList[1].code=13&d-148316-p=5&targetAudienceList[13].code=12&_targetAudienceList[13].selected=on&_targetAudienceList[8].selected=on&page=1&_targetAudienceList[7].selected=on&_targetAudienceList[0].selected=on&targetAudienceList[8].code=4&targetAudienceList[7].code=9&submit=Search&_targetAudienceList[5].selected=on&targetAudienceList[5].code=8&submit.y=10&keyword=&categoryId=2&submit.x=30&targetAudienceList[10].code=10&_targetAudienceList[2].selected=on&toDate=1%2F1%2F2018&targetAudienceList[0].code=7&_targetAudienceList[1].selected=on&targetAudienceList[12].code=2&targetAudienceList[3].code=14&_targetAudienceList[6].selected=on&targetAudienceList[9].code=11&fromDate=1%2F1%2F2017&targetAudienceList[11].code=6&targetAudienceList[4].code=5&campusCode=NB&targetAudienceList[2].code=1&targetAudienceList[6].code=3&_targetAudienceList[12].selected=on&_targetAudienceList[3].selected=on&_targetAudienceList[10].selected=on
for year in range(2011, 2018, 1):
    lisofresults = []
    startyear=year
    nextyear=year+1
    listofcolumnanme=['EventId','Title','CategoryId','Synopsis', 'Start_Time', 'End_Time', 'Location', 'Address', 'Campus', 'Room', 'City,_State,_Country', 'Fee', 'Sponsor', 'Category', 'Web_Site', 'Contact_Name', 'Contact_Email', 'Contact_Phone', 'Special_Criteria', 'Additional_Information', 'Target_Audience','Speaker','Status']
    currentdate=datetime.datetime.today().strftime('%m/%d/%Y')
    for categoryid in range(2, 17, 1):
        pagenumber=1
        if(categoryid!=13):
            pagenumber=1
            stillhaspage=True
            timetosleep=random.randrange(1, 5)
            print('time to sleep:',timetosleep)
            time.sleep(timetosleep)
            print("category id :"+str(categoryid))
            stillhaspage=True
            while(stillhaspage):
                #print(pagenumber)
                queryStringDictionary1={'region':'S','d-148316-p':pagenumber,'categoryId':categoryid,'submit.x':'30','campusCode':'NB','fromDate':'1/1/'+str(startyear),'toDate':'1/1/'+str(nextyear)}
                url1=urllib.parse.urlencode(queryStringDictionary1)
                url1='https://ruevents.rutgers.edu/events/displayEventList.html?'+url1  #.replace('+', '%20')
                #print(url1)
                pagenumber=pagenumber+1
                timetosleep=random.randrange(1, 6)
                print('time to sleep:',timetosleep)
                time.sleep(timetosleep)
              
                with urllib.request.urlopen(url1) as response:
                    html = response.read()
                    soup = BeautifulSoup(html, 'html.parser')
                    eventtables=soup.find("table", {"id": "event-list"})
                    if eventtables is None:
                        print("empty table")
                        #print(soup.prettify())
                        stillhaspage=False
                        break

                    if not eventtables is None:
                        rows = eventtables.find_all('tr')
                        #print(rows)
                        numberOfRow=0
                        listofhref= eventtables.find_all('a',  { "class" : "event-title" },href=True)
                        if(len(listofhref)==0):
                            stillhaspage=False
                            break
                        previouskeylength=len(listofcolumnanme)

                        for a in listofhref:
                            eventurl="https://ruevents.rutgers.edu/events/"+a['href']
                            parsed = urlparse(eventurl)
                            eventId=urllib.parse.parse_qs(parsed.query)['eventId']
                            #print(eventId[0])
                            time.sleep(0.5)
                            with urllib.request.urlopen(eventurl) as responseevent:
                                htmleventdetail = responseevent.read()
                                soup2 = BeautifulSoup(htmleventdetail, 'html.parser')
                                tableeventdetail=soup2.find_all("table", {"id": "event-details"})
                                results=dict((columnname,"") for columnname in listofcolumnanme)
                                eventtitle=soup2.find("h1",{ "class" : "title" } )
                                #print(eventtitle.string)
                                if any(currenteventdict['EventId'] == eventId[0] for currenteventdict in lisofresults):
                                    stillhaspage=False
                                    break
                                results['EventId']=eventId[0]
                                results['Title']=eventtitle.string
                                results['CategoryId']=str(categoryid)
                                for row in tableeventdetail[1].find_all('tr'):

                                    eventdetailhtml = row.find_all('td')
                                    if(len(eventdetailhtml)>0):
                                        if not eventdetailhtml[0].string is None:
                                            results[eventdetailhtml[0].string.replace(":","").replace(" ","_").strip()] = ""
                                        #print(eventdetailhtml[0].string.replace(":","").replace(" ","_"))
                                        if not eventdetailhtml[0].string is None and len(eventdetailhtml)>1 and not eventdetailhtml[1].string is None:
                                            cleaneventdeatil=re.sub('  +',' ',eventdetailhtml[1].string.replace("\t"," ").replace("\n","   ").strip()).strip()
                                            results[eventdetailhtml[0].string.replace(":","").replace(" ","_").strip()] = cleaneventdeatil
                                if(previouskeylength!=len(list(results.keys()))):
                                    print(len(list(results.keys())))
                                    print(list(results.keys()))
                                    print(url1)
                                else:
                                    lisofresults.append(results)
                                previouskeylength=len(list(results.keys()))

        with open('event_year_'+str(startyear)+'.csv', 'w') as f:  # Just use 'w' mode in 3.x
            w = csv.DictWriter(f, lisofresults[0].keys())
            w.writeheader()
            w.writerows(lisofresults)





time to sleep: 1
category id :2
time to sleep: 5
time to sleep: 3
time to sleep: 2
time to sleep: 1
time to sleep: 5
time to sleep: 1
time to sleep: 4
time to sleep: 4
time to sleep: 2
time to sleep: 3
time to sleep: 4
time to sleep: 4
time to sleep: 4
time to sleep: 5
time to sleep: 3
category id :3
time to sleep: 2
time to sleep: 4
time to sleep: 4
time to sleep: 1
time to sleep: 2
time to sleep: 5
time to sleep: 5
time to sleep: 5
time to sleep: 1
time to sleep: 3
time to sleep: 2
time to sleep: 1
time to sleep: 2
time to sleep: 5
time to sleep: 4
time to sleep: 2
time to sleep: 1
time to sleep: 1
time to sleep: 4
time to sleep: 5
time to sleep: 4
category id :4
time to sleep: 1
time to sleep: 2
time to sleep: 3
category id :5
time to sleep: 4
time to sleep: 2
time to sleep: 1
time to sleep: 3
time to sleep: 3
time to sleep: 3
time to sleep: 4
time to sleep: 3
time to sleep: 3
category id :6
time to sleep: 4
time to sleep: 5
time to sleep: 3
time to sleep: 1
time to sleep: 4
time to

time to sleep: 1
time to sleep: 2
time to sleep: 4
time to sleep: 3
time to sleep: 1
time to sleep: 1
time to sleep: 5
time to sleep: 1
time to sleep: 2
time to sleep: 3
time to sleep: 3
time to sleep: 1
time to sleep: 5
time to sleep: 3
time to sleep: 5
time to sleep: 3
time to sleep: 5
time to sleep: 1
category id :15
time to sleep: 4
time to sleep: 4
time to sleep: 2
time to sleep: 3
category id :16
time to sleep: 3
time to sleep: 5
time to sleep: 2
time to sleep: 4
time to sleep: 5
time to sleep: 1
time to sleep: 4
time to sleep: 1
time to sleep: 1
time to sleep: 2
category id :2
time to sleep: 3
time to sleep: 3
time to sleep: 3
time to sleep: 2
time to sleep: 4
time to sleep: 2
time to sleep: 3
time to sleep: 1
time to sleep: 5
time to sleep: 1
time to sleep: 5
time to sleep: 4
time to sleep: 2
time to sleep: 5
time to sleep: 4
time to sleep: 1
category id :3
time to sleep: 5
time to sleep: 3
time to sleep: 4
time to sleep: 3
time to sleep: 1
time to sleep: 3
time to sleep: 1
tim

time to sleep: 4
time to sleep: 1
time to sleep: 5
time to sleep: 4
time to sleep: 3
time to sleep: 5
time to sleep: 5
time to sleep: 3
time to sleep: 1
time to sleep: 3
time to sleep: 3
time to sleep: 5
time to sleep: 1
time to sleep: 2
time to sleep: 2
time to sleep: 2
time to sleep: 1
time to sleep: 4
time to sleep: 2
time to sleep: 4
time to sleep: 3
time to sleep: 5
time to sleep: 4
time to sleep: 1
time to sleep: 4
time to sleep: 1
time to sleep: 4
time to sleep: 4
time to sleep: 4
time to sleep: 5
time to sleep: 2
time to sleep: 2
time to sleep: 4
time to sleep: 2
time to sleep: 4
time to sleep: 5
time to sleep: 5
time to sleep: 2
time to sleep: 1
time to sleep: 2
time to sleep: 3
time to sleep: 2
time to sleep: 3
category id :15
time to sleep: 4
time to sleep: 1
time to sleep: 4
time to sleep: 5
time to sleep: 1
time to sleep: 3
category id :16
time to sleep: 5
time to sleep: 1
time to sleep: 1
time to sleep: 5
time to sleep: 4
time to sleep: 1
time to sleep: 1
time to sleep: 4

time to sleep: 1
time to sleep: 4
time to sleep: 4
time to sleep: 2
time to sleep: 5
time to sleep: 1
category id :14
time to sleep: 4
time to sleep: 1
time to sleep: 2
time to sleep: 1
time to sleep: 2
time to sleep: 4
time to sleep: 2
time to sleep: 5
time to sleep: 4
time to sleep: 1
time to sleep: 4
time to sleep: 3
time to sleep: 1
time to sleep: 2
time to sleep: 2
time to sleep: 5
time to sleep: 3
time to sleep: 1
time to sleep: 3
time to sleep: 3
time to sleep: 1
time to sleep: 5
time to sleep: 1
time to sleep: 1
time to sleep: 5
time to sleep: 3
time to sleep: 1
time to sleep: 1
time to sleep: 3
time to sleep: 4
time to sleep: 3
time to sleep: 5
time to sleep: 3
time to sleep: 4
time to sleep: 3
time to sleep: 2
time to sleep: 3
time to sleep: 2
time to sleep: 4
time to sleep: 2
time to sleep: 3
time to sleep: 4
time to sleep: 3
time to sleep: 2
time to sleep: 3
time to sleep: 1
time to sleep: 1
time to sleep: 4
time to sleep: 2
time to sleep: 2
time to sleep: 4
time to sleep: 

In [16]:
0

0